In [1]:
from algorithms.feature_extraction_loading import extract_diffusion_features

extract_diffusion_features({"davis": "tapvid_davis/tapvid_davis.pkl"}, enable_vae_slicing=False)

2024-07-12 11:47:12.190473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 11:47:12.206377: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 11:47:12.206403: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 11:47:12.217330: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 11:47:13.057532: W tensorflow/compiler/tf

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Dataset: davis


Progress: : 0it [00:00, ?it/s]

A goat climbs a rocky mountain ridge.


...

Progress: : 28it [08:52, 16.86s/it]

A man buying a television in the electronics store.


Progress: : 29it [09:07, 16.19s/it]

A woman on a horse jumps over an obstacle.


Progress: : 30it [09:21, 18.71s/it]


In [2]:
import os
os.chdir("ADL4CV")

FileNotFoundError: [Errno 2] No such file or directory: 'ADL4CV'

In [3]:
import pickle
import torch

# Replace 'your_file.pkl' with the path to your .pkl file
with open('output/features/davis/video_0.pkl', 'rb') as file:
    data = pickle.load(file)

feature_dict = data["features"]

for block_name, blocklist in feature_dict.items():
    print(block_name)
    for t in blocklist:
        print(t.shape)

encoder_block
torch.Size([90, 128, 128, 128])
torch.Size([90, 256, 64, 64])
torch.Size([90, 512, 32, 32])
torch.Size([90, 512, 32, 32])
up_block
torch.Size([90, 1280, 8, 8])
torch.Size([90, 1280, 16, 16])
torch.Size([90, 640, 32, 32])
torch.Size([90, 320, 32, 32])
down_block
torch.Size([90, 320, 16, 16])
torch.Size([90, 640, 8, 8])
torch.Size([90, 1280, 4, 4])
torch.Size([90, 1280, 4, 4])
mid_block
torch.Size([90, 1280, 4, 4])
decoder_block
torch.Size([90, 512, 64, 64])
torch.Size([90, 512, 128, 128])
torch.Size([90, 256, 256, 256])
torch.Size([90, 128, 256, 256])


In [4]:
# Create feature tensor for swan
import torch
import torch.nn.functional as func

f = feature_dict

feature = f["mid_block"][0].float() + f["down_block"][2].float() + f["down_block"][3].float() #1280x4x4

feature = func.interpolate(feature, scale_factor=2) +  f["up_block"][0].float() #1280x8x8

feature = func.interpolate(feature, scale_factor=2) +  f["up_block"][1].float() #1280x16x16


F,C,H,W = feature.shape
feature = feature.permute((0, 2, 3, 1)).contiguous().view(F*H,W,C)
feature = func.interpolate(feature, size=640)
feature = feature.view(F,H,W,-1).permute(0,3,1,2) #640x16x16

feature = feature +  func.interpolate(f["down_block"][1].float(), scale_factor=2) #640x16x16

F,C,H,W = feature.shape
feature = feature.permute((0, 2, 3, 1)).contiguous().view(F*H,W,C)
feature = func.interpolate(feature, size=320)
feature = feature.view(F,H,W,320).permute(0,3,1,2) #320x16x16

feature = feature + f["down_block"][0].float()

a1 = f["encoder_block"][2].float() + f["encoder_block"][3].float()
F,C,H,W = a1.shape
a1 = a1.permute((0, 2, 3, 1)).contiguous().view(F*H,W,C)
a1 = func.interpolate(a1, size=320)
a1 = a1.view(F,H,W,320).permute(0,3,1,2) #320x32x32

a2 = f["up_block"][2].float()
F,C,H,W = a2.shape
a2 = a2.permute((0, 2, 3, 1)).contiguous().view(F*H,W,C)
a2 = func.interpolate(a2, size=320)
a2 = a2.view(F,H,W,320).permute(0,3,1,2) #320x32x32

feature = func.interpolate(feature, scale_factor=2) + a1 + a2 #320x32x32

F,C,H,W = feature.shape
feature = feature.permute((0, 2, 3, 1)).contiguous().view(F*H,W,C)
feature = func.interpolate(feature, size=256)
feature = feature.view(F,H,W,256).permute(0,3,1,2) #256x32x32

a1 = f["decoder_block"][0].float()
F,C,H,W = a1.shape
a1 = a1.permute((0, 2, 3, 1)).contiguous().view(F*H,W,C)
a1 = func.interpolate(a1, size=256)
a1 = a1.view(F,H,W,256).permute(0,3,1,2) #256x64x64

feature = func.interpolate(feature, scale_factor=2) + a1 + f["encoder_block"][1].float() #256x64x64

F,C,H,W = feature.shape
feature = feature.permute((0, 2, 3, 1)).contiguous().view(F*H,W,C)
feature = func.interpolate(feature, size=128)
feature = feature.view(F,H,W,128).permute(0,3,1,2) #128x64x64

a1 = f["decoder_block"][1].float()
F,C,H,W = a1.shape
a1 = a1.permute((0, 2, 3, 1)).contiguous().view(F*H,W,C)
a1 = func.interpolate(a1, size=128)
a1 = a1.view(F,H,W,128).permute(0,3,1,2) #128x128x128

feature = func.interpolate(feature, scale_factor=2) + a1 + f["encoder_block"][0].float() #128x128x128

a1 = f["decoder_block"][2].float()
F,C,H,W = a1.shape
a1 = a1.permute((0, 2, 3, 1)).contiguous().view(F*H,W,C)
a1 = func.interpolate(a1, size=128)
a1 = a1.view(F,H,W,128).permute(0,3,1,2) #128x128x128

feature = func.interpolate(feature, scale_factor=2) + a1 + f["decoder_block"][3].float() #128x256x256

In [5]:
torch.save(feature, "goat_feature_tensor.pt")